In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
#Llamamos a la variable de entorno
DB_NAME = os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST= os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [4]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

#Creamos el cursor:
cur= conn.cursor()

In [5]:
#Comprobamos que la conexión está bien realizada:
cur.execute("SELECT version();")
cur.fetchone() #para que muestre el resultado de la query anterior

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

# Creación de los datos para las tablas:

In [6]:
#Importo los df procesados:
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS')
ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS =os.getenv('ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS')
ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS')
ARCHIVO_GUARDAR_DATOS_API_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_DATOS_API_PROCESADOS')
ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE')

## 1. Tabla pais_destino

### Unión de los paises y continentes tanto escrapeados como obtenidos de la API de turismo emisor

#### a) Turismo emisor

In [7]:
df_turismo_emisor_procesado=pd.read_csv(ARCHIVO_GUARDAR_DATOS_API_PROCESADOS)
df_turismo_emisor_procesado.columns

Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'CONTINENTE_DESTINO',
       'PAIS_DESTINO', 'TURISTAS', 'PERNOCTACIONES', 'ESTANCIA_MEDIA'],
      dtype='object')

In [8]:
df_turismo_emisor_procesado.head()

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucía,América,Argentina,"419,0","3616,0","8,6"
1,1,2019,7,Andalucía,América,Bolivia,"61,0","486,0","8,0"
2,2,2019,7,Andalucía,América,Brasil,"277,0","2214,0","8,0"
3,3,2019,7,Andalucía,América,Canadá,"483,0","6159,0","12,8"
4,4,2019,7,Andalucía,América,Chile,"242,0","2068,0","8,5"


In [9]:
df_paises_continentes_turismos_emisor=df_turismo_emisor_procesado[['PAIS_DESTINO','CONTINENTE_DESTINO']]
df_paises_continentes_turismos_emisor=df_paises_continentes_turismos_emisor.drop_duplicates()
df_paises_continentes_turismos_emisor.CONTINENTE_DESTINO.unique()

array(['América', 'Asia', 'Europa', 'Oceanía', 'África'], dtype=object)

In [10]:
df_paises_continentes_turismos_emisor.columns

Index(['PAIS_DESTINO', 'CONTINENTE_DESTINO'], dtype='object')

In [11]:
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.rename(
    columns={'PAIS_DESTINO':'nombre_pais_destino', 'CONTINENTE_DESTINO':'nombre_continente'})
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.sort_values(by='nombre_pais_destino').reset_index(drop=True)

In [12]:
df_paises_continentes_turismos_emisor

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Andorra,Europa
3,Angola,África
4,Antigua,América
...,...,...
153,Venezuela,América
154,Vietnam,Asia
155,Yemen,Asia
156,Zambia,África


#### b) Paises escrapeados

In [13]:
df_paises_continentes_escrapeo = pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS)
df_paises_continentes_escrapeo.columns

Index(['continente', 'pais', 'fecha_escrapeo', 'en_ultimo_escrapeo'], dtype='object')

In [14]:
df_paises_continentes_escrapeo.head()

,continente,pais,fecha_escrapeo,en_ultimo_escrapeo
0,Europa,Albania,2025-04-14,Si
1,Europa,Alemania,2025-04-14,Si
2,América,Antigua,2025-04-14,Si
3,América,Argentina,2025-04-14,Si
4,América,Aruba,2025-04-14,Si


In [15]:
df_paises_continentes_escrapeo=df_paises_continentes_escrapeo[['pais','continente']]
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.drop_duplicates()

In [16]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})

In [17]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.sort_values(by='nombre_pais_destino').reset_index(drop=True)
df_paises_continentes_escrapeo

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Antigua,América
3,Arabia Saudí,Asia
4,Argentina,América
...,...,...
120,Uruguay,América
121,Uzbekistán,Asia
122,Vietnam,Asia
123,Zanzíbar,África


#### c) Paises en itinerarios ciudades

In [18]:
df_itinerarios_ciudades = pd.read_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS)
df_itinerarios_ciudades.columns

Index(['itinerario_modificado_para_dividir', 'ciudad', 'pais_correcto'], dtype='object')

In [19]:
df_itinerarios_ciudades_pais_continentes = pd.DataFrame()
df_itinerarios_ciudades_pais_continentes['pais'] = df_itinerarios_ciudades['pais_correcto'].drop_duplicates().reset_index(drop=True)

In [20]:
df_itinerarios_ciudades_pais_continentes

,pais
0,Alemania
1,Argentina
2,Aruba
3,Australia
4,Austria
...,...
123,Macedonia del Norte
124,Bosnia y Herzegovina
125,Belice
126,Nigeria


In [21]:
df_opencage = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE)
df_opencage.columns

Index(['pais', 'continente_api', 'pais_api', 'latitud', 'longitud', 'ciudad'], dtype='object')

In [22]:
dict_paises_continentes_ciudades = {'pais':[], 'continente':[]}
for pais in df_itinerarios_ciudades_pais_continentes.pais:
    if pais in df_paises_continentes_escrapeo.nombre_pais_destino.tolist():
        pass
    else:
        if pais in df_paises_continentes_turismos_emisor.nombre_pais_destino.tolist():
           pass
        else:
           if pais in df_opencage.pais_api.tolist():
            #print(f'{pais} ya está en continentes opencage')
            dict_paises_continentes_ciudades['pais'].append(pais)
            continente = (df_opencage[df_opencage.pais_api == pais]).continente_api.unique()[0]
            dict_paises_continentes_ciudades['continente'].append(continente)
           else:
                print(f'{pais} ERROR')
df_paises_continentes_ciudades = pd.DataFrame(dict_paises_continentes_ciudades)
df_paises_continentes_ciudades

,pais,continente
0,Información desconocida,Información desconocida
1,Chequia,Europa


In [23]:
df_paises_continentes_ciudades= df_paises_continentes_ciudades.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})

In [24]:
df_paises_continentes_ciudades

,nombre_pais_destino,nombre_continente
0,Información desconocida,Información desconocida
1,Chequia,Europa


#### d) Uno los 3 DF para conseguir una única tabla con todos los paises y continentes

In [25]:
df_paises_continentes_totales=pd.concat(
    [df_paises_continentes_turismos_emisor,df_paises_continentes_escrapeo, df_paises_continentes_ciudades], axis=0)
df_paises_continentes_totales = df_paises_continentes_totales.drop_duplicates(['nombre_pais_destino', 'nombre_continente']).reset_index(drop=True)
df_paises_continentes_totales



,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Andorra,Europa
3,Angola,África
4,Antigua,América
...,...,...
184,Turks and Caicos,América
185,Turquía,Asia
186,Zanzíbar,África
187,Información desconocida,Información desconocida
